In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import itables

itables.init_notebook_mode()

def plot_demo(x, y):
    """
    plot the horn in 2D and 3D
    x and y are np.array in millimeters
    """

    # 2D Plot
    fig2d = go.Figure()
    fig2d.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Horn'))
    fig2d.update_layout(title='2D Horn Profile',
                        xaxis=dict(range=[0, None], scaleanchor='y'),
                        yaxis=dict(range=[0, None], scaleanchor='x'),
                        xaxis_title='x (mm)',
                        yaxis_title='Radius y (m)',
                        height=600, width=800)
    fig2d.show()

    # 3D wireframe (rotate profile)
    theta = np.linspace(0, 2*np.pi, 50)
    X = []
    Y = []
    Z = []

    for i in range(len(x)):
        X.extend([x[i]] * len(theta))
        Y.extend(y[i] * np.cos(theta))
        Z.extend(y[i] * np.sin(theta))

    fig3d = go.Figure(data=[go.Scatter3d(
        x=X, y=Y, z=Z,
        mode='lines+markers',
        marker=dict(size=1),
        name='3D Horn'
    )])

    fig3d.update_layout(title='3D Horn (Wireframe)',
                        scene=dict(
                            xaxis_title='x (mm)',
                            yaxis_title='y (mm)',
                            zaxis_title='z (mm)',
                            aspectmode="data",
                            aspectratio=dict(x=1, y=1, z=1),
                        ),
                        height=800, width=800)
    fig3d.show()


def generate_tractrix_horn(throat_radius, cutoff_freq, num_points=10):
    """
    throat_radius in mm
    cutoff freq in Hz
    num_points in number of points
    """
    throat_radius /= 1000  # convert to meters
    c = 343.0  # m/s
    a = c / (2 * np.pi * cutoff_freq)

    y = np.linspace(throat_radius, a, num_points)
    x = a * np.log((a + np.sqrt(a**2 - y**2)) / y) - np.sqrt(a**2 - y**2)

    df = pd.DataFrame({'x (m)': x, 'y (m)': y})

    df['x (mm)'] = df['x (m)'] * 1000
    df['y (mm)'] = df['y (m)'] * 1000

    plot_demo(df['x (mm)'], df['y (mm)'])

    return df[['x (mm)', 'y (mm)']]


def generate_spherical_horn(throat_radius, cutoff_freq, scale=4, fold=False, fold_back=True):
    """
    throat_radius in mm
    cutoff freq in Hz
    scale resolution in mm
    fold `True` for folding horn backward (default: False)
    fold_back `True` allow to fold back beyond the tweeter location (default: True)
    """
    throat_radius /= 1000  # convert to meters
    scale /= 1000
    c = 343.0  # m/s
    r0 = c / np.pi / cutoff_freq
    h0 = r0 - np.sqrt(r0*r0 - throat_radius*throat_radius)

    flare_rate = 4 * np.pi * cutoff_freq / c  # m

    x = np.arange(0, 1, scale)
    h = h0 * np.exp(flare_rate * x)
    xh = x - h + h0
    s = 2 * np.pi * r0 * h
    df = pd.DataFrame({'x': x, 'h': h, 'xh': xh, 's': s})
    if not fold:
        max_xh = df['xh'].max()
        max_x = df[df['xh'] == max_xh]['x'].max()
        df = df[df['x']<=max_x]
    else:
        df = df[df['s']/np.pi - df['h']**2 >= 0]
        if not fold_back:
            df = df[df['xh']>=0]

    df['y'] = np.sqrt(df['s'] / np.pi - df['h']**2)

    df['xh (mm)'] = df['xh'] * 1000
    df['y (mm)'] = df['y'] * 1000

    plot_demo(df['xh (mm)'], df['y (mm)'])

    return df[['xh (mm)', 'y (mm)']]


def generate_exponential_horn(throat_radius, cutoff_freq, scale=4):
    """
    throat_radius in mm
    cutoff freq in Hz
    scale resolution in mm
    """
    throat_radius /= 1000  # convert to meters
    scale /= 1000
    c = 343.0

    wave_length = c / cutoff_freq
    growth_factor = 4 * np.pi / wave_length

    x = np.arange(0, 1, scale)
    s = throat_radius**2 * np.pi * np.exp(growth_factor * x)
    r = np.sqrt(s/np.pi)
    cir = 2 * np.pi * r
    krm = cir / wave_length

    df = pd.DataFrame({'x':x, 'y': r, 'krm': krm})
    df = df[df['krm'] <= 1]

    df['x (mm)'] = df['x'] * 1000
    df['y (mm)'] = df['y'] * 1000

    plot_demo(df['x (mm)'], df['y (mm)'])

    return df[['x (mm)', 'y (mm)']]


In [ ]:
generate_tractrix_horn(15, 1500)

In [ ]:
generate_spherical_horn(15, 1500)

In [ ]:
generate_exponential_horn(8, 1000)